In [ ]:
import requests
import os
from dotenv import load_dotenv

def delete_cards_by_title(board_id, target_title="Standard:Reading"):
    """
    Delete all cards with a specific title from a Trello board.
    
    Args:
        board_id (str): The ID of the Trello board
        target_title (str): The title of cards to delete (default: "Standard:Reading")
    
    Returns:
        dict: Summary of deletion results
    """
    # Load environment variables from .env file
    load_dotenv()
    
    api_key = os.getenv("TRELLO_API_KEY")
    token = os.getenv("TRELLO_API_TOKEN")
    
    if not api_key:
        raise ValueError("TRELLO_API_KEY not found in environment variables")
    if not token:
        raise ValueError("TRELLO_API_TOKEN not found in environment variables")
    
    base_url = "https://api.trello.com/1"
    
    # Get all cards from the board
    cards_url = f"{base_url}/boards/{board_id}/cards"
    cards_params = {
        'key': api_key,
        'token': token,
        'fields': 'id,name,idList'
    }
    
    try:
        response = requests.get(cards_url, params=cards_params)
        response.raise_for_status()
        all_cards = response.json()
    except requests.RequestException as e:
        print(f"Error fetching cards: {e}")
        return {"success": False, "error": str(e)}
    
    # Filter cards with the target title
    cards_to_delete = [card for card in all_cards if card['name'] == target_title]
    
    print(f"Found {len(cards_to_delete)} cards with title '{target_title}'")
    
    if not cards_to_delete:
        print("No cards to delete.")
        return {"success": True, "deleted_count": 0, "cards_deleted": []}
    
    # Delete each matching card
    deleted_cards = []
    failed_deletions = []
    
    for card in cards_to_delete:
        card_id = card['id']
        card_name = card['name']
        
        delete_url = f"{base_url}/cards/{card_id}"
        delete_params = {
            'key': api_key,
            'token': token
        }
        
        try:
            delete_response = requests.delete(delete_url, params=delete_params)
            delete_response.raise_for_status()
            deleted_cards.append({"id": card_id, "name": card_name})
            print(f"✓ Deleted card: {card_name} (ID: {card_id})")
        except requests.RequestException as e:
            failed_deletions.append({"id": card_id, "name": card_name, "error": str(e)})
            print(f"✗ Failed to delete card: {card_name} (ID: {card_id}) - Error: {e}")
    
    # Summary
    results = {
        "success": len(failed_deletions) == 0,
        "deleted_count": len(deleted_cards),
        "failed_count": len(failed_deletions),
        "cards_deleted": deleted_cards,
        "failed_deletions": failed_deletions
    }
    
    print(f"\nSummary:")
    print(f"- Successfully deleted: {len(deleted_cards)} cards")
    print(f"- Failed deletions: {len(failed_deletions)} cards")
    
    return results

# Usage example:
# Replace 'your_board_id_here' with your actual board ID
board_id = os.getenv("TRELLO_JOURNAL_BOARD_ID")
# Delete all cards with title "Standard:Reading"
results = delete_cards_by_title(board_id, "OnFoot")

# Print final results
print("\n" + "="*50)
print("FINAL RESULTS:")
print(f"Operation successful: {results['success']}")
print(f"Cards deleted: {results['deleted_count']}")
if results['failed_count'] > 0:
    print(f"Failed deletions: {results['failed_count']}")
    for failure in results['failed_deletions']:
        print(f"  - {failure['name']} (ID: {failure['id']}): {failure['error']}")